In [17]:
import pandas as pd
import numpy as np
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
import os
import pyranges as pr
import gseapy as gp
from gseapy import barplot, dotplot
import json

In [18]:
cell_type = 'CD8'
de_type = 'include_emergent_disappear' # include_emergent_disappear, no_emergent_include_disappear, no_emergent_no_disappear
fig_path = '/home/che/TRIM/git/tcr/figures/saliency'
parent_path = f'{fig_path}/{de_type}/{cell_type}/data'
geneset_path = f'{parent_path}/gene_sets'
csv_path = f'{parent_path}/diffexp_df.csv'
output_path = f'{parent_path}/compiled'
venn_path = f'{output_path}/venn/all_gradient'

if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(venn_path):
    os.makedirs(venn_path)

print(output_path)
print(venn_path)

/home/che/TRIM/git/tcr/figures/saliency/include_emergent_disappear/CD8/data/compiled
/home/che/TRIM/git/tcr/figures/saliency/include_emergent_disappear/CD8/data/compiled/venn/all_gradient


In [19]:
# Read in the whole dataset
df = pd.read_csv(csv_path)
df.rename(columns={'Unnamed: 0': 'Gene'}, inplace=True)
df

,Gene,lfc_post,lfc_pre,lfc_clone_pre_10,lfc_clone_pre_1,lfc_post_pvals_adj,lfc_pre_pvals_adj,lfc_clone_pre_10_pvals_adj,lfc_clone_pre_1_pvals_adj,salient_genes,salient_genes_rank,lfc_post_rank,lfc_pre_rank,lfc_clone_pre_10_rank,lfc_clone_pre_1_rank
0,A1BG,0.841133,-0.794820,-0.725561,-1.166667,9.641343e-08,0.00015,1.121982e-12,2.611040e-41,-0.000007,10791.0,1865.0,17173.0,16486.0,17158.0
1,A1BG-AS1,0.630395,-0.090250,-0.484623,0.291439,1.000000e+00,1.00000,1.000000e+00,1.000000e+00,0.000031,2010.0,2357.0,13270.0,15601.0,6076.0
2,A2M,-1.642448,2.252429,-0.178504,-0.586164,1.000000e+00,1.00000,1.000000e+00,1.000000e+00,0.000051,658.0,18341.0,454.0,13850.0,13951.0
3,A2M-AS1,0.501576,0.178894,0.893563,0.957310,1.000000e+00,1.00000,1.000000e+00,1.000000e+00,-0.000027,15070.0,2735.0,8739.0,3887.0,2239.0
4,A4GALT,-1.497684,2.251524,0.973174,-0.587068,NaN,NaN,NaN,NaN,0.000005,6178.0,17228.0,1452.5,2731.5,14951.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19251,ZXDC,0.052903,-0.004820,0.134014,-0.018931,1.000000e+00,1.00000,1.000000e+00,1.000000e+00,0.000010,5125.0,6347.0,11984.0,9515.0,10352.0
19252,ZYG11A,-1.497684,0.396684,2.828014,1.267772,NaN,1.00000,1.000000e+00,1.000000e+00,-0.000029,15255.0,17228.0,6003.0,359.0,1522.0
19253,ZYG11B,0.317693,-0.155747,-0.495282,-0.672724,1.000000e+00,1.00000,1.000000e+00,1.000000e+00,-0.000004,10006.0,3636.0,14064.0,15651.0,16224.0
19254,ZYX,-0.018861,-0.026237,0.016083,0.089319,1.000000e+00,1.00000,1.000000e+00,1.000000e+00,-0.000028,15111.0,7555.0,12323.0,11443.0,8987.0


In [20]:
# read in the whole gene list
with open('/home/che/TRIM/data_processed/combined_data_columns.npz', 'rb') as f:
    npzfile = np.load(f, allow_pickle=True)
    combined_data_columns = npzfile['cols']
complete_gene_list = combined_data_columns.tolist()
len(complete_gene_list)

29750

In [21]:
# first subset to the genes found by different methods
if de_type == 'no_emergent_no_disappear':
    alpha = 0.05
    log2fc_threshold = -np.inf
    high_gradient_threshold = 0.996
    low_gradient_threshold = 1- high_gradient_threshold #0.01
else:
    alpha = 0.01
    log2fc_threshold = -np.inf
    high_gradient_threshold = 0.99
    low_gradient_threshold = 1- high_gradient_threshold #0.01

top_genes_lfc_post = df[df['lfc_post_pvals_adj'] < alpha].copy()
top_genes_lfc_post = top_genes_lfc_post[abs(top_genes_lfc_post['lfc_post']) > log2fc_threshold]
df['top_genes_lfc_post'] = df['Gene'].isin(top_genes_lfc_post['Gene'])
print('len top_genes_lfc_post', len(top_genes_lfc_post))

top_genes_lfc_pre = df[df['lfc_pre_pvals_adj'] < alpha].copy()
top_genes_lfc_pre = top_genes_lfc_pre[abs(top_genes_lfc_pre['lfc_pre']) > log2fc_threshold]
df['top_genes_lfc_pre'] = df['Gene'].isin(top_genes_lfc_pre['Gene'])
print('len top_genes_lfc_pre', len(top_genes_lfc_pre))

top_genes_grad = df[(df['salient_genes'] > df['salient_genes'].quantile(high_gradient_threshold)) | 
                    (df['salient_genes'] < df['salient_genes'].quantile(low_gradient_threshold))]
df['top_genes_grad'] = df['Gene'].isin(top_genes_grad['Gene'])
print('len top_genes_grad:', len(top_genes_grad))

top_genes_lfc_clone_pre_1 = df[df['lfc_clone_pre_1_pvals_adj'] < alpha]
top_genes_lfc_clone_pre_1 = top_genes_lfc_clone_pre_1[abs(top_genes_lfc_clone_pre_1['lfc_clone_pre_1']) > log2fc_threshold]
df['top_genes_lfc_clone_pre_1'] = df['Gene'].isin(top_genes_lfc_clone_pre_1['Gene'])
print('len top_genes_lfc_clone_pre_1', len(top_genes_lfc_clone_pre_1))

top_genes_lfc_clone_pre_10 = df[df['lfc_clone_pre_10_pvals_adj'] < alpha]
top_genes_lfc_clone_pre_10 = top_genes_lfc_clone_pre_10[abs(top_genes_lfc_clone_pre_10['lfc_clone_pre_10']) > log2fc_threshold]
df['top_genes_lfc_clone_pre_10'] = df['Gene'].isin(top_genes_lfc_clone_pre_10['Gene'])
print('len top_genes_lfc_clone_pre_10', len(top_genes_lfc_clone_pre_10))

df_subset = df[df['Gene'].isin(top_genes_lfc_post['Gene']) | 
                df['Gene'].isin(top_genes_lfc_pre['Gene']) |
               df['Gene'].isin(top_genes_grad['Gene']) | 
               df['Gene'].isin(top_genes_lfc_clone_pre_10['Gene']) | 
               df['Gene'].isin(top_genes_lfc_clone_pre_1['Gene'])].copy()
df_subset

len top_genes_lfc_post 432
len top_genes_lfc_pre 466
len top_genes_grad: 386
len top_genes_lfc_clone_pre_1 1519
len top_genes_lfc_clone_pre_10 888


,Gene,lfc_post,lfc_pre,lfc_clone_pre_10,lfc_clone_pre_1,lfc_post_pvals_adj,lfc_pre_pvals_adj,lfc_clone_pre_10_pvals_adj,lfc_clone_pre_1_pvals_adj,salient_genes,salient_genes_rank,lfc_post_rank,lfc_pre_rank,lfc_clone_pre_10_rank,lfc_clone_pre_1_rank,top_genes_lfc_post,top_genes_lfc_pre,top_genes_grad,top_genes_lfc_clone_pre_1,top_genes_lfc_clone_pre_10
0,A1BG,0.841133,-0.794820,-0.725561,-1.166667,9.641343e-08,1.504766e-04,1.121982e-12,2.611040e-41,-0.000007,10791.0,1865.0,17173.0,16486.0,17158.0,True,True,False,True,True
37,ABCB9,-0.504494,-1.068248,-0.942749,-1.228584,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-0.000089,19153.0,14188.0,17735.0,17094.0,17257.0,False,False,True,False,False
45,ABCD2,0.612471,-0.986414,-0.716174,-0.660411,1.000000e+00,1.000000e+00,1.000000e+00,5.904538e-03,-0.000063,18558.0,2407.0,17579.0,16456.0,16187.0,False,False,False,True,False
64,ABHD17A,-0.496471,0.581175,0.784946,1.030809,2.574536e-03,9.659899e-08,3.985006e-25,1.507854e-35,0.000059,393.0,14141.0,4818.0,4126.0,2057.0,True,True,False,True,True
76,ABI3,-0.405581,0.551165,0.528436,1.170311,6.399717e-02,5.022182e-05,3.270137e-07,3.755932e-33,-0.000041,16932.0,13530.0,4982.0,5072.0,1738.0,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19162,ZNF814,0.311633,-1.714411,-1.153668,-1.398973,1.000000e+00,1.000000e+00,1.000000e+00,1.870137e-05,-0.000070,18788.0,3671.0,18472.0,17487.0,17462.0,False,False,False,True,False
19199,ZNHIT1,-0.080221,0.239010,0.256901,0.390924,1.000000e+00,1.000000e+00,1.000000e+00,6.796115e-03,0.000015,4331.0,8756.0,7871.0,7497.0,4917.0,False,False,False,True,False
19215,ZRANB2-AS1,-1.497684,4.351961,-1.127263,-2.687505,NaN,1.000000e+00,1.000000e+00,1.000000e+00,0.000075,158.0,17228.0,51.0,17451.0,18660.0,False,False,True,False,False
19222,ZSCAN18,0.466925,-0.539419,-1.463122,-1.888522,1.000000e+00,1.000000e+00,2.669569e-05,3.251631e-18,0.000020,3514.0,2869.0,16469.0,17919.0,17843.0,False,False,False,True,True


In [22]:
# read in gencode file
protein_gene_path = '/home/che/TRIM/git/tcr/other_information/gencode.v41.annotation.gtf'
gr = pr.read_gtf(protein_gene_path)
gencode_df = gr.df
gencode_df = gencode_df[gencode_df['Feature'] == 'gene']
gencode_df = gencode_df[gencode_df['gene_type'] == 'protein_coding']
protein_coding_genes = gencode_df['gene_name'].unique().tolist()
print('len protein_coding_genes:', len(protein_coding_genes))

len protein_coding_genes: 19991


In [23]:
# protein coding genes downloaded from: https://www.genenames.org/download/statistics-and-files/
protein_gene_path = '/home/che/TRIM/git/tcr/other_information/gene_with_protein_product.txt'
protein_genes = pd.read_csv(protein_gene_path, sep='\t')
protein_genes_2 = protein_genes['symbol'].unique().tolist()
print('len protein_genes_2:', len(protein_genes_2))
len(set(protein_coding_genes) - set(protein_genes_2))

/tmp/ipykernel_720346/2460345661.py:3: DtypeWarning: Columns (35,38,45,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  protein_genes = pd.read_csv(protein_gene_path, sep='\t')


len protein_genes_2: 19268


987

In [24]:
# Subset to human protein-coding genes
df_subset = df_subset[df_subset['Gene'].isin(protein_coding_genes)]
df_subset

,Gene,lfc_post,lfc_pre,lfc_clone_pre_10,lfc_clone_pre_1,lfc_post_pvals_adj,lfc_pre_pvals_adj,lfc_clone_pre_10_pvals_adj,lfc_clone_pre_1_pvals_adj,salient_genes,salient_genes_rank,lfc_post_rank,lfc_pre_rank,lfc_clone_pre_10_rank,lfc_clone_pre_1_rank,top_genes_lfc_post,top_genes_lfc_pre,top_genes_grad,top_genes_lfc_clone_pre_1,top_genes_lfc_clone_pre_10
0,A1BG,0.841133,-0.794820,-0.725561,-1.166667,9.641343e-08,1.504766e-04,1.121982e-12,2.611040e-41,-0.000007,10791.0,1865.0,17173.0,16486.0,17158.0,True,True,False,True,True
37,ABCB9,-0.504494,-1.068248,-0.942749,-1.228584,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-0.000089,19153.0,14188.0,17735.0,17094.0,17257.0,False,False,True,False,False
45,ABCD2,0.612471,-0.986414,-0.716174,-0.660411,1.000000e+00,1.000000e+00,1.000000e+00,5.904538e-03,-0.000063,18558.0,2407.0,17579.0,16456.0,16187.0,False,False,False,True,False
64,ABHD17A,-0.496471,0.581175,0.784946,1.030809,2.574536e-03,9.659899e-08,3.985006e-25,1.507854e-35,0.000059,393.0,14141.0,4818.0,4126.0,2057.0,True,True,False,True,True
76,ABI3,-0.405581,0.551165,0.528436,1.170311,6.399717e-02,5.022182e-05,3.270137e-07,3.755932e-33,-0.000041,16932.0,13530.0,4982.0,5072.0,1738.0,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19134,ZNF776,0.372331,-1.575151,-1.176315,-1.623787,1.000000e+00,1.000000e+00,1.000000e+00,2.243293e-03,-0.000053,18020.0,3317.0,18361.0,17529.0,17655.0,False,False,False,True,False
19162,ZNF814,0.311633,-1.714411,-1.153668,-1.398973,1.000000e+00,1.000000e+00,1.000000e+00,1.870137e-05,-0.000070,18788.0,3671.0,18472.0,17487.0,17462.0,False,False,False,True,False
19199,ZNHIT1,-0.080221,0.239010,0.256901,0.390924,1.000000e+00,1.000000e+00,1.000000e+00,6.796115e-03,0.000015,4331.0,8756.0,7871.0,7497.0,4917.0,False,False,False,True,False
19222,ZSCAN18,0.466925,-0.539419,-1.463122,-1.888522,1.000000e+00,1.000000e+00,2.669569e-05,3.251631e-18,0.000020,3514.0,2869.0,16469.0,17919.0,17843.0,False,False,False,True,True


In [25]:
top_genes_lfc_post = set(df_subset[df_subset['top_genes_lfc_post'] == True]['Gene'].tolist())
top_genes_lfc_pre = set(df_subset[df_subset['top_genes_lfc_pre'] == True]['Gene'].tolist())
top_genes_grad = set(df_subset[df_subset['top_genes_grad'] == True]['Gene'].tolist())
top_genes_lfc_clone_pre_10 = set(df_subset[df_subset['top_genes_lfc_clone_pre_10'] == True]['Gene'].tolist())
top_genes_lfc_clone_pre_1 = set(df_subset[df_subset['top_genes_lfc_clone_pre_1'] == True]['Gene'].tolist())

len(top_genes_lfc_post), len(top_genes_lfc_pre), \
        len(top_genes_lfc_clone_pre_10), len(top_genes_lfc_clone_pre_1), \
                len(top_genes_grad)

(401, 431, 808, 1408, 219)

### Now, perform gene set enrichment analysis for each list

- gene_list: important genes found by each method
- backgroun: all genes in the genome (measured in RNA-seq)

In [26]:
# Helper functions to remove some gene sets with high overlap --------
from itertools import combinations
def jaccard_similarity(set1, set2):
    """Calculate Jaccard similarity between two sets"""
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0

def filter_enrichr_by_jaccard_similarity(enr, threshold=0.5, column_name='Genes', 
                                         sort_by='Adjusted P-value', ascending=True):
    """
    Filter Enrichr results to keep only gene sets with minimal overlap
    
    Parameters:
    -----------
    enr : pandas.DataFrame
        Enrichr results dataframe containing gene sets
    threshold : float, default=0.5
        Maximum allowed Jaccard similarity between kept gene sets
    column_name : str, default='Genes'
        Column name containing the genes (typically semicolon or comma separated)
    sort_by : str, default='Adjusted P-value'
        Column to sort by before filtering
    ascending : bool, default=True
        Whether to sort in ascending order
        
    Returns:
    --------
    pandas.DataFrame
        Filtered Enrichr results with minimal gene set overlap
    """
    # Make a copy to avoid modifying the original
    df = enr.copy()
    
    # Convert gene lists to sets
    gene_sets = []
    for gene_list in df[column_name]:
        # Handle different separators (comma, semicolon, etc.)
        if isinstance(gene_list, str):
            if ';' in gene_list:
                genes = set(gene_list.split(';'))
            elif ',' in gene_list:
                genes = set(gene_list.split(','))
            else:
                genes = set(gene_list.split())
            gene_sets.append(genes)
        else:
            gene_sets.append(set())
    
    # Sort by specified column (usually p-value or adjusted p-value)
    df['gene_sets'] = gene_sets
    df = df.sort_values(by=sort_by, ascending=ascending).reset_index(drop=True)
    
    # Initialize list to keep track of which rows to keep
    to_keep = [True] * len(df)
    
    # Compare each gene set with all preceding gene sets that we've decided to keep
    for i in range(1, len(df)):
        if not to_keep[i]:  # Skip if already marked for removal
            continue
            
        for j in range(i):
            if not to_keep[j]:  # Skip if already marked for removal
                continue
                
            # Calculate Jaccard similarity
            similarity = jaccard_similarity(df.loc[i, 'gene_sets'], df.loc[j, 'gene_sets'])
            
            # If similarity exceeds threshold, mark the current gene set for removal
            if similarity > threshold:
                to_keep[i] = False
                break
            # else:
            #     print(f"Gene set {i} and {j} have Jaccard similarity: {similarity:.2f}")
    
    # Filter the dataframe
    filtered_df = df[to_keep].copy()
    
    # Remove the temporary gene_sets column
    filtered_df.drop(columns=['gene_sets'], inplace=True)
    
    return filtered_df

In [27]:
methods_list = [top_genes_grad, top_genes_lfc_post, top_genes_lfc_pre, top_genes_lfc_clone_pre_1, top_genes_lfc_clone_pre_10]
methods_name = ['top_genes_grad', 'top_genes_lfc_post', 'top_genes_lfc_pre', 'top_genes_lfc_clone_pre_1', 'top_genes_lfc_clone_pre_10']
geneset_enrich_list = []

for top_genes_temp in methods_list:
    gene_list_temp = list(top_genes_temp)
    background_genes = complete_gene_list

    # Run enrichment analysis
    enr = gp.enrichr(
            gene_list=gene_list_temp,
            gene_sets='GO_Biological_Process_2018', 
            organism='Human',
            background=background_genes,
            outdir=None,
            cutoff=0.05,
        )
    enr_df_temp = enr.results
    enr_df_temp.to_csv(f'{geneset_path}/{methods_name[methods_list.index(top_genes_temp)]}_enrichment.csv', index=False)
    enr_df_temp = enr_df_temp[enr_df_temp['Adjusted P-value'] < 0.1]
    
    filtered_enr = filter_enrichr_by_jaccard_similarity(enr_df_temp, threshold=0.5)
    # save filtered_enr in geneset_path
    filtered_enr.to_csv(f'{geneset_path}/{methods_name[methods_list.index(top_genes_temp)]}_filtered_enrichment.csv', index=False)
    # get the top 15 list of genesets with adjusted p-value < 0.1
    geneset_enrich_temp = filtered_enr['Term'].tolist()[:15]
    geneset_enrich_list.append(geneset_enrich_temp)

In [28]:
# flatten geneset_enrich_list
geneset_enrich_list_flat = [item for sublist in geneset_enrich_list for item in sublist]
geneset_enrich_list_flat = list(set(geneset_enrich_list_flat))
len(geneset_enrich_list_flat)
# save geneset_enrich_list_flat in geneset_path
with open(f'{geneset_path}/geneset_enrich_list.json', 'w') as f:
    json.dump(geneset_enrich_list_flat, f)

In [29]:
geneset_enrich_list_flat

['positive regulation of cell proliferation (GO:0008284)',
 'nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)',
 'SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
 'positive regulation of peptidyl-tyrosine phosphorylation (GO:0050731)',
 'platelet degranulation (GO:0002576)',
 'T cell receptor signaling pathway (GO:0050852)',
 'T cell activation (GO:0042110)',
 'ribosomal small subunit biogenesis (GO:0042274)',
 'regulation of apoptotic process (GO:0042981)',
 'regulation of small GTPase mediated signal transduction (GO:0051056)',
 'glomerular epithelial cell differentiation (GO:0072311)',
 'immunological synapse formation (GO:0001771)',
 'regulation of cell proliferation (GO:0042127)',
 'ribosomal large subunit biogenesis (GO:0042273)',
 'regulation of immune response (GO:0050776)',
 'ribosome assembly (GO:0042255)',
 'neutrophil degranulation (GO:0043312)',
 'cellular response to interferon-gamma (GO:0071346)',
 'cytokine-medi

In [30]:
# Download the GO:BP library from Enrichr (as a dictionary)
go_bp_dict = gp.get_library(name='GO_Biological_Process_2018', organism='Human')

In [31]:
go_enrichr_all = {}
for go in geneset_enrich_list_flat:
    go_enrichr_all[go] = go_bp_dict[go]

In [32]:
with open(f'{geneset_path}/go_enrichr.json', 'w') as f:
    json.dump(go_enrichr_all, f, indent=4)